In [1]:
!pip install xlrd

In [2]:
import pandas as pd

In [3]:
training=pd.read_csv(r'C:\Users\admin\Documents\PREPARATION\GiveMeSomeCredit\cs-training.csv')
training.head(3)
training.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 12 columns):
 #   Column                                Non-Null Count   Dtype  
---  ------                                --------------   -----  
 0   Unnamed: 0                            150000 non-null  int64  
 1   SeriousDlqin2yrs                      150000 non-null  int64  
 2   RevolvingUtilizationOfUnsecuredLines  150000 non-null  float64
 3   age                                   150000 non-null  int64  
 4   NumberOfTime30-59DaysPastDueNotWorse  150000 non-null  int64  
 5   DebtRatio                             150000 non-null  float64
 6   MonthlyIncome                         120269 non-null  float64
 7   NumberOfOpenCreditLinesAndLoans       150000 non-null  int64  
 8   NumberOfTimes90DaysLate               150000 non-null  int64  
 9   NumberRealEstateLoansOrLines          150000 non-null  int64  
 10  NumberOfTime60-89DaysPastDueNotWorse  150000 non-null  int64  
 11  

In [4]:
age_bins = pd.cut(training['age'], bins=5)
age_bins

0         (43.6, 65.4]
1         (21.8, 43.6]
2         (21.8, 43.6]
3         (21.8, 43.6]
4         (43.6, 65.4]
              ...     
149995    (65.4, 87.2]
149996    (43.6, 65.4]
149997    (43.6, 65.4]
149998    (21.8, 43.6]
149999    (43.6, 65.4]
Name: age, Length: 150000, dtype: category
Categories (5, interval[float64, right]): [(-0.109, 21.8] < (21.8, 43.6] < (43.6, 65.4] < (65.4, 87.2] < (87.2, 109.0]]

In [5]:
training['NumberOfDependents_missing'] = training['NumberOfDependents'].isnull().astype(int)
training['NumberOfDependents'] = training.groupby(age_bins)['NumberOfDependents'].transform(lambda x:x.fillna(x.median()))

C:\Users\admin\AppData\Local\Temp\ipykernel_22952\1544995388.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  training['NumberOfDependents'] = training.groupby(age_bins)['NumberOfDependents'].transform(lambda x:x.fillna(x.median()))


In [6]:
training['MonthlyIncome_missing'] = training['MonthlyIncome'].isnull().astype(int)
training['MonthlyIncome'] = training.groupby(age_bins)['MonthlyIncome'].transform(lambda x:x.fillna(x.median()))

C:\Users\admin\AppData\Local\Temp\ipykernel_22952\1240742365.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  training['MonthlyIncome'] = training.groupby(age_bins)['MonthlyIncome'].transform(lambda x:x.fillna(x.median()))


In [7]:
# Tổng số lần trễ hạn
training['TotalDelinquency'] = (
    training['NumberOfTime30-59DaysPastDueNotWorse'] +
    training['NumberOfTime60-89DaysPastDueNotWorse'] +
    training['NumberOfTimes90DaysLate']
)

In [8]:
training['SeriousDlqin2yrs'].value_counts(normalize=True)

SeriousDlqin2yrs
0    0.93316
1    0.06684
Name: proportion, dtype: float64

In [9]:
# Xác định biến mục tiêu và đặc trưng
X = training.drop('SeriousDlqin2yrs', axis=1)
y = training['SeriousDlqin2yrs']

# 1.4 Chia train/test, giữ tỷ lệ lớp (stratify)
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

In [13]:
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, roc_auc_score

# SMOTE áp dụng trên dữ liệu chưa scale
sm = SMOTE(random_state=42)
X_train_res, y_train_res = sm.fit_resample(X_train, y_train)

# Huấn luyện lại model
model = RandomForestClassifier(random_state=42)
model.fit(X_train_res, y_train_res)

# Dự đoán
y_pred = model.predict(X_test)
y_proba = model.predict_proba(X_test)[:, 1]

# Đánh giá
print(classification_report(y_test, y_pred))
print("ROC AUC:", roc_auc_score(y_test, y_proba))


              precision    recall  f1-score   support

           0       0.96      0.94      0.95     27995
           1       0.34      0.44      0.38      2005

    accuracy                           0.90     30000
   macro avg       0.65      0.69      0.67     30000
weighted avg       0.92      0.90      0.91     30000

ROC AUC: 0.8367716269248295


In [15]:
import xgboost as xgb
from sklearn.metrics import classification_report, roc_auc_score
import joblib 

# Tính tỷ lệ mất cân bằng
scale_pos_weight = y_train.value_counts()[0] / y_train.value_counts()[1]

# Khởi tạo mô hình XGBoost
model = xgb.XGBClassifier(
    use_label_encoder=False,
    eval_metric='logloss',
    scale_pos_weight=scale_pos_weight,
    max_depth=5,
    n_estimators=300,
    learning_rate=0.1,
    random_state=42
)

# Train
model.fit(X_train, y_train)
joblib.dump(model, 'xgb_model.pkl')

# Dự đoán
y_proba = model.predict_proba(X_test)[:, 1]
y_pred_custom = (y_proba >= 0.75).astype(int)

# Đánh giá
from sklearn.metrics import classification_report, roc_auc_score
print(classification_report(y_test, y_pred_custom))
print("ROC AUC:", roc_auc_score(y_test, y_proba))


c:\Users\admin\anaconda3\envs\churn_env\lib\site-packages\xgboost\training.py:183: UserWarning: [23:08:03] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


              precision    recall  f1-score   support

           0       0.96      0.94      0.95     27995
           1       0.38      0.52      0.44      2005

    accuracy                           0.91     30000
   macro avg       0.67      0.73      0.70     30000
weighted avg       0.93      0.91      0.92     30000

ROC AUC: 0.8617370576060296
